# 1. Import demanding packages

In [1]:
import ee
import geemap
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 2. Authenticate geemap

In [2]:
ee.Authenticate()

True

In [3]:
geemap.set_proxy(port=7890)

In [4]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# 3. Define functioins of BAESCM

In [5]:
def mask_s2_clouds(image):
  """ Mask clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

In [6]:
def merge_clusters(df):
    """
    Merge clusters based on clusterID and record the merging process.

    Args:
    - df: A DataFrame containing cluster information.
      Must include:
        - ClusterID
        - Cluster center values (mean values of each band).
        - Cluster size (total pixel number in the cluster).
        - Sample count (total number of samples in the cluster).

    Returns:
    - df: Merged DataFrame with updated clusterID and merge history.
    """
    df['clusterID'] = df['clusterID'].astype(int).astype(str)  # Convert clusterID column to string
    df['merged_clusters'] = df['clusterID'].apply(lambda x: [x])  # Add a column to record the merging process, initial value is clusterID

    centers_columns = ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12", "NDVI", "lda"]  # All bands used for clustering

    while True:
        # Find clusters with sample count less than 2
        small_clusters = df[df['sample_count'] < 2]

        if small_clusters.empty:
            # All clusters have sample count >= 2, exit
            break

        for _, small_cluster in small_clusters.iterrows():
            # Get the clusterID of the current small cluster
            small_cluster_id = small_cluster['clusterID']

            # Calculate the distance from the current cluster to other clusters' centers
            other_clusters = df[df['clusterID'] != small_cluster_id]

            distances = np.linalg.norm(
                other_clusters[centers_columns].values - small_cluster[centers_columns].values.astype(np.float64), axis=1
            )
            # Find the nearest cluster
            closest_index = np.argmin(distances)
            closest_cluster = other_clusters.iloc[closest_index]
            closest_cluster_id = closest_cluster['clusterID']

            # Merge the small cluster into the nearest cluster
            total_size = small_cluster['size'] + closest_cluster['size']
            total_sample_count = small_cluster['sample_count'] + closest_cluster['sample_count']

            total_num0 = small_cluster['num0'] + closest_cluster['num0']
            total_num1 = small_cluster['num1'] + closest_cluster['num1']
            total_a11 = small_cluster['a11'] + closest_cluster['a11']
            total_a12 = small_cluster['a12'] + closest_cluster['a12']
            total_a21 = small_cluster['a21'] + closest_cluster['a21']
            total_a22 = small_cluster['a22'] + closest_cluster['a22']
            total_sample_count0 = small_cluster['sample_count0'] + closest_cluster['sample_count0']
            total_sample_count1 = small_cluster['sample_count1'] + closest_cluster['sample_count1']

            # Calculate the new cluster center (weighted average)
            new_center = (small_cluster[centers_columns] * small_cluster['size'] +
                          closest_cluster[centers_columns] * closest_cluster['size']) / total_size

            # Update the target cluster information
            df.loc[df['clusterID'] == closest_cluster_id, centers_columns] = new_center.values
            df.loc[df['clusterID'] == closest_cluster_id, 'size'] = total_size
            df.loc[df['clusterID'] == closest_cluster_id, 'sample_count'] = total_sample_count

            df.loc[df['clusterID'] == closest_cluster_id, 'num0'] = total_num0
            df.loc[df['clusterID'] == closest_cluster_id, 'num1'] = total_num1
            df.loc[df['clusterID'] == closest_cluster_id, 'a11'] = total_a11
            df.loc[df['clusterID'] == closest_cluster_id, 'a12'] = total_a12
            df.loc[df['clusterID'] == closest_cluster_id, 'a21'] = total_a21
            df.loc[df['clusterID'] == closest_cluster_id, 'a22'] = total_a22
            df.loc[df['clusterID'] == closest_cluster_id, 'sample_count0'] = total_sample_count0
            df.loc[df['clusterID'] == closest_cluster_id, 'sample_count1'] = total_sample_count1

            # Update the merging history
            merged_history = df.loc[df['clusterID'] == closest_cluster_id, 'merged_clusters'].values[0]
            updated_history = merged_history + [small_cluster_id]
            # Get the target row index
            target_index = df[df['clusterID'] == closest_cluster_id].index[0]
            # Use .at to ensure updating the specific cell
            df.at[target_index, 'merged_clusters'] = updated_history

            # Delete the current small cluster
            df = df[df['clusterID'] != small_cluster_id].reset_index(drop=True)

            # Exit the loop and recheck due to index changes
            break
    return df

In [7]:
def process_matrix(df):
    """
    Correct the confusion matrix constructed from non-probabilistic samples.

    Args:
    -df: DataFrame containing the confusion matrix to be corrected and layer weights.
    
    Returns:
    -df: DataFrame with corrected values.
    """
    file_matrix = df[['clusterID', 'a11', 'a12', 'a21', 'a22', 'num0', 'num1', 'size', 'sampling_var']].copy()

    # Apply weighted correction to non-target layers. If no samples fall into this layer, assign 0.5 initially, then apply weight to ensure the final confusion matrix sums to 1.
    file_matrix['p11'] = file_matrix.apply(lambda row: 0.5 * row['num0'] / row['size'] if row['a11'] == 0 and row['a12'] == 0 
                                            else row['a11'] / (row['a11'] + row['a12']) * row['num0'] / row['size'], axis=1)
    file_matrix['p12'] = file_matrix.apply(lambda row: 0.5 * row['num0'] / row['size'] if row['a11'] == 0 and row['a12'] == 0 
                                            else row['a12'] / (row['a11'] + row['a12']) * row['num0'] / row['size'], axis=1)

    # Apply weighted correction to target layers. If no samples fall into this layer, assign 0.5 initially, then apply weight to ensure the final confusion matrix sums to 1.
    file_matrix['p21'] = file_matrix.apply(lambda row: 0.5 * row['num1'] / row['size'] if row['a21'] == 0 and row['a22'] == 0 
                                            else row['a21'] / (row['a21'] + row['a22']) * row['num1'] / row['size'], axis=1)
    file_matrix['p22'] = file_matrix.apply(lambda row: 0.5 * row['num1'] / row['size'] if row['a21'] == 0 and row['a22'] == 0 
                                            else row['a22'] / (row['a21'] + row['a22']) * row['num1'] / row['size'], axis=1)

    return file_matrix

In [8]:
def compute_cluster_percentage(feature):
    """
    Calculate the number of different cluster categories within each county.

    Args:
    -feature: subregional feature

    Returns:
    -feature: subregional feature with cluster component
    
    """
    histogram = cluster_result.reduceRegion(
        reducer=ee.Reducer.fixedHistogram(0, cluster_num, cluster_num),
        geometry=feature.geometry(),
        scale=10,
        crs='EPSG:32615',
        maxPixels=1e11
    ).get('cluster')  # Get the histogram result

    # Extract the histogram data
    key_list = ee.List(ee.Array(histogram).slice(1, 0, 1).toList().flatten())  # First column: category index
    value_list = ee.List(ee.Array(histogram).slice(1, 1, 2).toList().flatten())  # Second column: category count

    # Convert category indexes to strings
    string_list = key_list.map(lambda value: ee.String(ee.Number(value).int().format()))

    # Create a dictionary {category: count}
    dictionary_from_list = ee.Dictionary.fromLists(string_list, value_list)

    # Add the dictionary as properties to the feature
    return feature.set(dictionary_from_list)

In [9]:
def compute_cluster_statistics(c):
    """
    Calculate the number of classified target class pixels in each cluster.
    
    Args:
    -c: cluster ID

    Returns:
    -feature: component (classified target class and non-target class) in each cluster
    
    """
    c = ee.Number(c)

    # Generate a mask for each cluster
    mask_value = cluster_result.eq(c)

    # Calculate the classification histogram for each cluster
    cluster_histo = cla.updateMask(mask_value).reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=state.geometry(),
        scale=10,
        crs='EPSG:32615',
        maxPixels=1e11
    )

    # Extract the "classification" field from the histogram
    histogram_dict = ee.Dictionary(cluster_histo.get("classification"))

    # Get the counts for category 0 and category 1
    num0 = ee.Number(histogram_dict.get('0', 0))
    num1 = ee.Number(histogram_dict.get('1', 0))

    # Create a Feature containing the cluster ID and counts
    feature = ee.Feature(
        ee.Geometry.Point([115.11564806768662, 38.63820782099307]), 
        {'clusterID': c, 'num0': num0, 'num1': num1}
    )

    return feature

In [10]:
def compute_classified_statistics(feature):
    """ 
    Calculate the pixel number of classified target class for each subregion 

    Args:
    -feature: subregion feature

    Returns:
    -feature: subregion feature containing classified target and non-target pixel numbers
    """
    classified_histo = cla.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=feature.geometry(),
        scale=10,
        crs='EPSG:32615',
        maxPixels=1e11
    ).get('classification')
    
    # Extract values from the histogram
    dict_ = ee.Dictionary(classified_histo)
    num0 = dict_.get('0', 0)
    num1 = dict_.get('1', 0)
    
    # Return the feature with added statistics
    return feature.set('num0', num0, 'num1', num1)

In [11]:
def compute_cluster_mean(c):
    """
    Calculate the mean spectral values for each cluster

    Args: 
    -c: cluster ID

    Returns:
    -feature: features containing mean spectral values for each cluster
    """
    c = ee.Number(c)

    # Create a mask for each cluster
    mask_value = cluster_result.eq(c)

    # Calculate the mean value of each band for the current cluster
    mean_values = dataset.addBands([normalized_lda]).updateMask(mask_value).reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=state.geometry(),
        scale=10,
        maxPixels=1e11
    )

    # Extract the computed mean values as a dictionary
    mean_dict = ee.Dictionary(mean_values)

    # Extract the mean for each band
    band_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'NDVI', 'lda']
    band_means = {band: mean_dict.get(band, 0) for band in band_names}

    # Create a Feature containing the clusterID and band means
    feature = ee.Feature(
        ee.Geometry.Point([115.11564806768662, 38.63820782099307]),
        {'clusterID': c, **band_means}
    )

    return feature

In [12]:
def compute_error_matrix(c):
    """
    Calculate the error matrix for each cluster

    Args: 
    -c: cluster ID

    Returns:
    -feature: feature contains error matrix for each cluster
    
    """
    c = ee.Number(c)

    # Compute the error matrix for the current cluster
    confusion_matrix = (
        vali_sample.filterMetadata('cluster', 'equals', c)  # Filter sample for the current cluster
        .errorMatrix('classification', 'type', [0, 1])  # Compute the confusion matrix (true/false positives/negatives)
        .array()  # Convert to an array
        .toList()  # Convert the array to a list
        .flatten()  # Flatten the list
    )

    # Extract the four elements of the confusion matrix
    a11 = ee.Number(confusion_matrix.get(0))  # True Positive
    a12 = ee.Number(confusion_matrix.get(1))  # False Positive
    a21 = ee.Number(confusion_matrix.get(2))  # False Negative
    a22 = ee.Number(confusion_matrix.get(3))  # True Negative

    # Create a Feature containing the clusterID and confusion matrix values
    feature = ee.Feature(
        ee.Geometry.Point([115.11564806768662, 38.63820782099307]),
        {
            'clusterID': c,
            'a11': a11,
            'a12': a12,
            'a21': a21,
            'a22': a22,
        }
    )

    return feature

# 4. Import inputs of BAESCM

In [13]:
"""
The inputs of BAESCM must contain:

1. The boundary of global region
    description: FeatureCollection
    
2. The boundary of subregions
    description: FeatureCollection
    
3. The samples for area estimation
    description: FeatureCollection. The sample file should contains two fields: "classification" and "type". 
                 "classification" field refers to the map class of the sample. It's also the stratification of sampling. 1 means target class and 0 means non-target class
                 "type" field refers to the true class of the sample. 1 means target class and 0 means non-target class.
4. The Sentinel-2 images of the global region
    description: ImageCollection. The bands should contain 'B2','B3','B4','B5','B6','B7','B8','B8A','B11' and 'B12'.
    
5. The classification map
    description: Image. 1 means target class and 0 means non-target class.
"""

# The boundary of global region. We take Nebraska state as an example here.
state = ee.FeatureCollection("users/bz327603/nebraska_state")

# The boundary of subregions. We take counties of Nebraska state as an example here.
county = ee.FeatureCollection("users/bz327603/nebraska_county")

# The samples for area estimation. We take 500 samples (250 samples of mapped target class and 250 samples of mapped non-taget class) in total as an example.
sample = ee.FeatureCollection("users/bz327603/nebraska_sample500")

# The classification map of the global region for area estimation. We take a RF map as an example.
cla = ee.Image("users/bz327603/nebraska_classified")

# The Sentinel-2 images 
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2019-01-01', '2019-12-31')
    .filter(ee.Filter.calendarRange(200,240))
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    .map(mask_s2_clouds)
    .filterBounds(state.geometry())
    .select(['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12'])
    .mean()
    .unmask(0)
    .clip(state)
)

In [14]:
"""
Due to computational limitations of the GEE platform, some calculations cannot be completed online.
The subsequent processing includes exporting the computation results to your legacy assets, you need to specify your legacy assets directory.
For example, 'users/bz327603'
"""
asset_directory = 'users/bz327603'

# 5. Set parameters of BAESCM

In [15]:
"""
The parameters of BAESCM contain:
1. The cluster number of K-Means.
2. The z value. The z quartile corresponding to the required confidence level of confidence interval for area estimations. 
"""

# cluster number for K-Means clustering
cluster_num = 6

# z value for prior confidence of area estimations. We assumed that there is a 99% probability that the estimated values for all subregions are correct, so the corresponding z quantile equals 2.58
prior = 2.58

# z value for your required confidence of confidence interval. For example, we want to derive 95% confidence intervals of the area estimations, so the corresponding z quantile equals 1.96
z = 1.96

# 6. Data Process

## 6.1 FLDA transfer

In [16]:
# Calculate NDVI bands and add to dataset

# calculate NDVI
redBand = dataset.select('B4') 
nirBand = dataset.select('B8')
ndvi = nirBand.subtract(redBand).divide(nirBand.add(redBand)).rename('NDVI');

# add NDVI to dataset
dataset = dataset.addBands(ndvi);

In [22]:
# Export samples for FLDA component 

# stratification sampling
flda_sample = dataset.addBands(cla).stratifiedSample(
    numPoints=1000,          
    classBand='classification',  # stratification
    seed=123,                
    region=state,           
    scale=10,               
    geometries=True         
)

# start a task to export
task = ee.batch.Export.table.toAsset(
    collection=flda_sample,
    description='nebraska_flda_sample',
    assetId=asset_directory+'/nebraska_flda_sample'
)
task.start()

print("The task is running, please check in the Task Manager and wait until it is completed")

The task is running, please check in the Task Manager and wait until it is completed


In [17]:
# import FLDA samples
flda_sample = ee.FeatureCollection(asset_directory+"/nebraska_flda_sample")

In [19]:
# prepare inputs of FLDA algorithm 
df = geemap.ee_to_df(flda_sample)
X = df[['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12','NDVI']].to_numpy()
Y = df[['classification']].to_numpy().ravel()

# fit a FLDA model
clf = LinearDiscriminantAnalysis()
clf.fit(X, Y)

# derive transfer factors
bar = clf.xbar_.tolist()
scale = clf.scalings_.flatten().tolist()

# FLDA transfer and derive FLDA component
bandNames = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12', 'NDVI']
lda = dataset.select(bandNames) \
    .subtract(bar) \
    .multiply(scale) \
    .reduce(ee.Reducer.sum())

# calculate 2% and 98% percentiles as minimum and maximum values of FLDA component
percentileValue = lda.clip(state).reduceRegion(
    reducer=ee.Reducer.percentile([2, 98]),   
    geometry=state.geometry(),               
    scale=10,                                
    crs='EPSG:32615',                        
    maxPixels=1e11                           
)

blank = ee.Feature(ee.Geometry.Point([115.11564806768662, 38.63820782099307])).set({
    'min': percentileValue.getNumber('sum_p2'),
    'max': percentileValue.getNumber('sum_p98')
})

# export to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([blank]),
    description='minmax',
    fileFormat='CSV'
)
task.start()

print("The task is running, please check in the Task Manager and wait until it is completed, then update your min and max values of the following cell!")

The task is running, please check in the Task Manager and wait until it is completed, then update your min and max values of the following cell!


In [20]:
min_value = ee.Number(-2.726414338)   # replace your 2% percentile here
max_value = ee.Number(2.691629333) # replace your 98% percentile here

# normalize FLDA with min and max values
normalized_lda = lda.subtract(min_value).divide(max_value.subtract(min_value)).rename('lda').toFloat()

## 6.2 K-Means clustering

In [26]:
# training samples for K-Means
training = (dataset.addBands([normalized_lda])
                .sample(region=state,  
                        scale=10,      
                        tileScale=16,  
                        numPixels=20000))  

# train a K-Means model
clusterer = (ee.Clusterer.wekaKMeans(nClusters=cluster_num)
                 .train(training))

# derive K-Means results
cluster_result = dataset.addBands([normalized_lda]).cluster(clusterer)

# export K-Means results to asset
task = ee.batch.Export.image.toAsset(
    image = cluster_result,
    description = 'nebraska_cluster',
    assetId = asset_directory+'/nebraska_cluster', 
    region = state.geometry(),
    scale = 10,
    maxPixels = 1e11
)

task.start()

print("The task is running, please check in the Task Manager and wait until it is completed")

The task is running, please check in the Task Manager and wait until it is completed


In [21]:
#import cluster result
cluster_result = ee.Image(asset_directory+"/nebraska_cluster")

## 6.3 Area Estimation

In [22]:
# Calculate the cluster percentage for all counties
cluster_percent = county.map(compute_cluster_percentage)

df_cluster_percent = geemap.ee_to_df(cluster_percent)

In [23]:
# create a list from 0 to cluster_num-1 
cluster_list = ee.List.sequence(0, cluster_num - 1, 1)

In [24]:
# Calculate statistics for all clusters
statistic = cluster_list.map(compute_cluster_statistics)

# Convert the list of statistics to a FeatureCollection
cluster_gram = ee.FeatureCollection(statistic.flatten())

df_cluster_gram = geemap.ee_to_df(cluster_gram)

In [25]:
# Apply the statistical calculation to each county
classified_statistic = county.map(compute_classified_statistics)

# Convert the FeatureCollection to a DataFrame
df_cla_percent = geemap.ee_to_df(classified_statistic)

# Calculate the classification percentage (num1 / (num0 + num1))
df_cla_percent['classify_percent'] = df_cla_percent['num1'] / (df_cla_percent['num0'] + df_cla_percent['num1'])

In [32]:
# Generate a sequence from 0 to cluster_num-1
cluster_list = ee.List.sequence(0, cluster_num - 1, 1)

# Compute statistics (mean) for all clusters
mean_by_cluster = cluster_list.map(compute_cluster_mean)

# Convert the result to a FeatureCollection
mean_gram = ee.FeatureCollection(mean_by_cluster.flatten())

# Export to Google Drive
export_task = ee.batch.Export.table.toDrive(
    collection=mean_gram,
    description="cluster_mean",
    fileFormat="CSV"
)

# Start the export task
export_task.start()

print("Export task started. Check the GEE Task Manager for progress and wait until it is completed. Then download it from Google Drive to your computer!")

Export task started. Check the GEE Task Manager for progress and wait until it is completed. Then download it from Google Drive to your computer!


In [26]:
# import the cluster mean file, update the following directory with yours
file_cluster_mean = pd.read_csv(r"C:\Users\bo'zhang\Downloads\cluster_mean.csv")

In [27]:
# samples for calculating error matrix of each cluster
vali_sample = cluster_result.sampleRegions(
    collection=sample,
    properties=sample.first().propertyNames(),  # Retain original properties
    scale=10,  # Resolution of the image
    tileScale=16
)


# Calculate the error matrix for all clusters
cluster_error_matrix = cluster_list.map(compute_error_matrix)

# Convert the list to a FeatureCollection
out_collection = ee.FeatureCollection(cluster_error_matrix.flatten())

# Convert FeatureCollection to a pandas DataFrame for analysis
df_cluster_error_matrix = geemap.ee_to_df(out_collection)

In [28]:
# Number of samples in each cluster that belong to the non-target class
df_cluster_error_matrix['sample_count0'] = df_cluster_error_matrix['a11']+df_cluster_error_matrix['a12'] 

# Number of samples in each cluster that belong to the target class
df_cluster_error_matrix['sample_count1'] = df_cluster_error_matrix['a21'] + df_cluster_error_matrix['a22'] 

# Total number of samples in each cluster
df_cluster_error_matrix['sample_count'] = df_cluster_error_matrix['sample_count0'] + df_cluster_error_matrix['sample_count1'] 

# Total pixel count in each cluster
df_cluster_gram['size'] = df_cluster_gram['num0']+df_cluster_gram['num1'] 

cluster_mean = file_cluster_mean[["clusterID","B2","B3","B4","B5","B6","B7","B8","B8A","B11","B12","NDVI","lda"]]

# Merge cluster composition, confusion matrix, and cluster center data
join_lines = pd.merge(pd.merge(df_cluster_gram,df_cluster_error_matrix,on='clusterID',how='left'),cluster_mean,on='clusterID',how='left') 
# join_lines columns: [["clusterID","num0","num1","size","a11","a12","a21","a22","sample_count0","sample_count1","sample_count","B2","B3","B4","B5","B6","B7","B8","B8A","B11","B12","NDVI","lda"]]

# Merged clusters with their attributes
merged_df = merge_clusters(join_lines) 

# print(merged_df)

# Calculate the random variance for each cluster
merged_df['wh0'] = merged_df['num0']/merged_df['size']
merged_df['wh1'] = merged_df['num1'] / merged_df['size']
merged_df['s0_square'] = merged_df['a11']/merged_df['sample_count0']*(merged_df['a12']/merged_df['sample_count0'])*merged_df['sample_count0']/(merged_df['sample_count0']-1)
merged_df['s1_square'] = merged_df['a21']/merged_df['sample_count1']*(merged_df['a22']/merged_df['sample_count1'])*merged_df['sample_count1']/(merged_df['sample_count1']-1)
merged_df['sampling_var'] = merged_df.apply(lambda row: 1/row['sample_count']*(row['wh0']*row['s0_square']+row['wh1']*row['s1_square'])+1/(row['sample_count']*row['sample_count'])*((1-row['wh0'])*row['s0_square']+(1-row['wh1'])*row['s1_square']) if row['sample_count0']>=2 and row['sample_count1']>=2
                                            else (row['a11']+row['a21'])/row['sample_count']*(1-((row['a11']+row['a21'])/row['sample_count']))/ (row['sample_count']-1),axis=1)

# Used to store the merged count of each cluster in each county
percent_df = df_cluster_percent[['obj_id']].copy() 

# Loop through the merged_df's merged_clusters column, merge the corresponding columns from df_cluster_percent, and store in percent_df
for _, row in merged_df.iterrows():
    cluster_id = row['merged_clusters'][0]        # Use the first element of merged_clusters as the new column name
    cluster_ids_to_merge = row['merged_clusters']
    percent_df[cluster_id] = df_cluster_percent[cluster_ids_to_merge].sum(axis=1) # Merge corresponding columns

# print(percent_df)

# Calculate the proportion of each cluster in each county
columns_to_normalize = percent_df.columns[1:] # Columns to be normalized
percent_df['total'] = percent_df[columns_to_normalize].sum(axis=1) # Sum the columns to be normalized as the denominator for normalization

# Create a DataFrame containing only obj_id
region_df = percent_df[['obj_id']].copy() 

# Add the proportion of each cluster to region_df
for col in columns_to_normalize:
    region_df[f'{col}_percent'] = percent_df[col] / percent_df['total']

# Adjust the format of region_df to create region_long
region_long = pd.melt(
    region_df,
    id_vars = ['obj_id'],
    var_name = 'cluster_column',
    value_name = 'weight'
)
# Extract clusterID as the key for joining tables
region_long['clusterID'] = region_long['cluster_column'].str.extract(r'(\d+)')

# Correct the confusion matrix
corrected_df = process_matrix(merged_df)

# Merge the two DataFrames
weighted_df = pd.merge(region_long, corrected_df, on='clusterID')


# Apply weighted confusion matrix and calculate the random variance for each cluster
weighted_df['weighted_p11'] = weighted_df['weight'] * weighted_df['p11']
weighted_df['weighted_p12'] = weighted_df['weight'] * weighted_df['p12']
weighted_df['weighted_p21'] = weighted_df['weight'] * weighted_df['p21']
weighted_df['weighted_p22'] = weighted_df['weight'] * weighted_df['p22']
weighted_df['total_sampling_var'] = weighted_df['weight']**2 * weighted_df['sampling_var']

# Group by obj_id and sum the results to get result_df
county_matrix_df = weighted_df.groupby('obj_id').agg({
    'weighted_p11': 'sum',
    'weighted_p12': 'sum',
    'weighted_p21': 'sum',
    'weighted_p22': 'sum',
    'total_sampling_var': 'sum'
}).reset_index()

# Classification area for each county
county_area = df_cla_percent[["obj_id","classify_percent"]] 

# Merge the corrected confusion matrix with classification area for each county
result_df = pd.merge(county_matrix_df, county_area, on='obj_id', how='left') 

# Corrected area
result_df['corrected_percent'] = result_df['classify_percent'] + result_df['weighted_p12'] - result_df['weighted_p21'] 

# Force corrected area to be 0 if it's less than 0
result_df['corrected_percent'] = result_df['corrected_percent'].apply(lambda x: max(0, x)) 

# Calculate downscale variance
result_df['down_variance'] = ((result_df['weighted_p12'] - result_df['weighted_p21']).abs() / prior) ** 2 

# Calculate total standard deviation
result_df['total_std'] = np.sqrt(result_df['total_sampling_var'] + result_df['down_variance']) 

# Calculate the left bound of the confidence interval
result_df['left_bound'] = result_df['corrected_percent'] - z * result_df['total_std'] 
result_df['left_bound'] = result_df['left_bound'].apply(lambda x: max(0, x)) 

# Calculate the right bound of the confidence interval
result_df['right_bound'] = result_df['corrected_percent'] + z * result_df['total_std'] 
result_df['right_bound'] = result_df['right_bound'].apply(lambda x: min(1, x))

# output of BAESCM
print(result_df[['obj_id','corrected_percent','left_bound','right_bound']])

# 'obj_id' refers to the subregion ID, 'corrected_percent' refers to the corrected area of the subregion
# 'left_bound' and 'right_bound' refer to the left and right boundaries of confidence interval


    obj_id  corrected_percent  left_bound  right_bound
0        0           0.167419    0.099206     0.235633
1        1           0.215345    0.146918     0.283773
2        2           0.001924    0.000000     0.010934
3        3           0.004057    0.000000     0.026352
4        4           0.000000    0.000000     0.021265
..     ...                ...         ...          ...
88      88           0.272830    0.205506     0.340154
89      89           0.326350    0.240652     0.412048
90      90           0.137460    0.078875     0.196046
91      91           0.049617    0.007245     0.091990
92      92           0.174855    0.102265     0.247445

[93 rows x 4 columns]


In [29]:
import openpyxl
result_df.to_excel("output.xlsx", index=False) 